**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [22]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
import cv2
import json
import skvideo
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
skvideo.setFFmpegPath('C:/ffmpeg/bin')


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

```epsilon``` is essential for exploration, if we don't use epsilon the algorith will only exploit what it learns without discovering the map. We only use it to train because we don't want to take random decision while testing.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.1
epochs_train = 100 # set small when debugging
epochs_test = 30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` is a grid with a 1 at the mouse's position of the mouse and ```board``` is a grid with that represents the cheese and the poison as reward. Both grids are use to represent a state (a 5*5 portion is extracted from both grids)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,4) # Return a random action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs + 1):
        
        ##### FILL IN HERE
        
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=False)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix + str(e))
        
        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 3.5/12.0. Average score (-8.5)
Win/lose count 5.0/8.0. Average score (-5.75)
Win/lose count 3.5/7.0. Average score (-5.0)
Win/lose count 3.0/5.0. Average score (-4.25)
Win/lose count 1.5/8.0. Average score (-4.7)
Win/lose count 2.5/8.0. Average score (-4.833333333333333)
Win/lose count 6.5/11.0. Average score (-4.785714285714286)
Win/lose count 2.5/4.0. Average score (-4.375)
Win/lose count 7.0/8.0. Average score (-4.0)
Win/lose count 1.5/5.0. Average score (-3.95)
Win/lose count 1.5/13.0. Average score (-4.636363636363637)
Win/lose count 2.0/4.0. Average score (-4.416666666666667)
Win/lose count 2.5/7.0. Average score (-4.423076923076923)
Win/lose count 2.5/4.0. Average score (-4.214285714285714)
Win/lose count 2.5/3.0. Average score (-3.966666666666667)
Win/lose count 2.5/14.0. Average score (-4.4375)
Win/lose count 0.5/4.0. Average score (-4.382352941176471)
Win/lose count 3.0/6.0. Average score (-4.305555555555555)
Win/lose count 4.0/7.0. Average score (-4.2368421052

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0] # If the memory is full, remove the oldest transition

    def random_access(self):
        rd_index = np.random.randint(len(self.memory)) # Get a random index
        return self.memory[rd_index]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_]) # Memorize the last transition
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            transition = self.memory.random_access() # Get a random memorized transition
            
            input_states[i] = transition[0]
            current_q_list = self.model.predict(np.expand_dims(transition[0], axis=0)) # Predict Q value of current state

            if game_over_:
                ######## FILL IN
                new_q_value = transition[3] # Learned Q value. No prediction of the next move Q value because the game is over
                
            else:
                ######## FILL IN
                future_q_list = self.model.predict(np.expand_dims(transition[1], axis=0)) # Predict Q value of future state
                new_q_value = transition[3] + self.discount * np.max(future_q_list) # Learned Q value
                
            ######## FILL IN  
            target_q[i] = current_q_list
            target_q[i][transition[2]] = new_q_value # Update Q value
            
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q) # Train the batch


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        ###### FILL IN
        # Fully connected dense network
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Dense(100))
        model.add(Dense(4))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train' + str(epochs_train) + '.mp4'))

Epoch 001/100 | Loss 1.9375 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/100 | Loss 2.0430 | Win/lose count 4.0/7.0 (-3.0)
Epoch 003/100 | Loss 2.0038 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/100 | Loss 1.9541 | Win/lose count 2.0/5.0 (-3.0)
Epoch 005/100 | Loss 2.1140 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/100 | Loss 1.8884 | Win/lose count 6.5/4.0 (2.5)
Epoch 007/100 | Loss 1.8847 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/100 | Loss 2.0820 | Win/lose count 2.0/1.0 (1.0)
Epoch 009/100 | Loss 2.0076 | Win/lose count 11.5/7.0 (4.5)
Epoch 010/100 | Loss 2.0729 | Win/lose count 3.5/3.0 (0.5)
Epoch 011/100 | Loss 1.9982 | Win/lose count 12.0/6.0 (6.0)
Epoch 012/100 | Loss 2.0594 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/100 | Loss 1.9773 | Win/lose count 7.5/3.0 (4.5)
Epoch 014/100 | Loss 1.9960 | Win/lose count 7.0/2.0 (5.0)
Epoch 015/100 | Loss 2.0087 | Win/lose count 12.5/1.0 (11.5)
Epoch 016/100 | Loss 1.9195 | Win/lose count 5.0/2.0 (3.0)
Epoch 017/100 | Loss 2.0521 | Win/lose count 9

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        # NN Model        
        ####### FILL IN
        
        # First convolutional layer
        model = Sequential()
        model.add(Conv2D(64, (2, 2), data_format="channels_last", input_shape=(5, 5, self.n_state)))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        
        # Second convolutional layer
        model.add(Conv2D(64, (2, 2), data_format="channels_last", input_shape=(4, 4, self.n_state)))
        model.add(Flatten())
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        
        # Dense layer
        model.add(Dense(2048))
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model



In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train' + str(epochs_train) + '.mp4'))

Epoch 001/100 | Loss 0.0102 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/100 | Loss 0.0275 | Win/lose count 5.5/1.0 (4.5)
Epoch 003/100 | Loss 0.0128 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/100 | Loss 0.0115 | Win/lose count 2.0/3.0 (-1.0)
Epoch 005/100 | Loss 0.0063 | Win/lose count 3.0/7.0 (-4.0)
Epoch 006/100 | Loss 0.0101 | Win/lose count 5.5/2.0 (3.5)
Epoch 007/100 | Loss 0.0052 | Win/lose count 0/5.0 (-5.0)
Epoch 008/100 | Loss 0.0085 | Win/lose count 5.5/3.0 (2.5)
Epoch 009/100 | Loss 0.0184 | Win/lose count 3.5/1.0 (2.5)
Epoch 010/100 | Loss 0.0139 | Win/lose count 4.0/4.0 (0.0)
Epoch 011/100 | Loss 0.0215 | Win/lose count 2.0/3.0 (-1.0)
Epoch 012/100 | Loss 0.0244 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/100 | Loss 0.0187 | Win/lose count 3.5/4.0 (-0.5)
Epoch 014/100 | Loss 0.0486 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/100 | Loss 0.0343 | Win/lose count 4.0/1.0 (3.0)
Epoch 016/100 | Loss 0.0309 | Win/lose count 4.0/1.0 (3.0)
Epoch 017/100 | Loss 0.0269 | Win/lose count 3.5/1.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.5/0. Average score (5.5)
Win/lose count 5.0/0. Average score (5.25)
Win/lose count 3.0/0. Average score (4.5)
Win/lose count 5.0/0. Average score (4.625)
Win/lose count 11.5/0. Average score (6.0)
Win/lose count 6.0/0. Average score (6.0)
Win/lose count 6.0/0. Average score (6.0)
Win/lose count 1.0/0. Average score (5.375)
Win/lose count 3.0/0. Average score (5.111111111111111)
Win/lose count 1.0/0. Average score (4.7)
Win/lose count 3.0/0. Average score (4.545454545454546)
Win/lose count 15.5/0. Average score (5.458333333333333)
Win/lose count 10.5/0. Average score (5.846153846153846)
Win/lose count 2.5/0. Average score (5.607142857142857)
Win/lose count 1.0/0. Average score (5.3)
Win/lose count 3.0/0. Average score (5.15625)
Win/lose count 5.5/1.0. Average score (5.117647058823529)
Win/lose count 1.5/0. Average score (4.916666666666667)
Win/lose count 2.5/0. Average score (4.7894736842105265)
Win/lose count 4.5/0. Average score (4.775)
Win/lose count 

In [16]:
HTML(display_videos('cnn_test' + str(epochs_test) + '.mp4'))

In [17]:
HTML(display_videos('fc_test' + str(epochs_test) + '.mp4'))

We observe that the mouse stays in always in the same part of the map (even with the $\epsilon$-greedy strategy)

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In my solution, the epsilon decreases linearly with respect to the number of epochs thanks to a recurrent expression

In [18]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True) # add a parameter train to desactivate malus position while testing

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Decrease epsilon
        agent.set_epsilon(agent.epsilon * (epoch - e) / (epoch - e + 1))
        
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, round(win, 1), round(lose, 1), round(win-lose, 1)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) # Create a grid for malus_position

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
              
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y] # Add a malus if the postion has already been visited
              
        self.malus_position[self.x, self.y] = 0.1 # Put a malus on the current position
              
        self.t = self.t + 1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2) # Add the malus position grid to describe the state
              
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore' + str(epochs_train) + '.mp4'))

Epoch 001/100 | Loss 0.0185 | Win/lose count 11.0/20.0 (-9.0)
Epoch 002/100 | Loss 0.0088 | Win/lose count 7.0/21.7 (-14.7)
Epoch 003/100 | Loss 0.0144 | Win/lose count 1.7/21.0 (-19.3)
Epoch 004/100 | Loss 0.0160 | Win/lose count 1.7/20.5 (-18.8)
Epoch 005/100 | Loss 0.0130 | Win/lose count 7.2/18.6 (-11.4)
Epoch 006/100 | Loss 0.0127 | Win/lose count 5.7/22.6 (-16.9)
Epoch 007/100 | Loss 0.0159 | Win/lose count 5.7/20.5 (-14.8)
Epoch 008/100 | Loss 0.0147 | Win/lose count 10.2/21.3 (-11.1)
Epoch 009/100 | Loss 0.0182 | Win/lose count 7.4/20.0 (-12.6)
Epoch 010/100 | Loss 0.0181 | Win/lose count 9.9/17.1 (-7.2)
Epoch 011/100 | Loss 0.0193 | Win/lose count 10.2/19.4 (-9.2)
Epoch 012/100 | Loss 0.0726 | Win/lose count 10.4/19.4 (-9.0)
Epoch 013/100 | Loss 0.1277 | Win/lose count 18.2/22.3 (-4.1)
Epoch 014/100 | Loss 0.0906 | Win/lose count 7.6/19.2 (-11.6)
Epoch 015/100 | Loss 0.0817 | Win/lose count 16.8/17.9 (-1.1)
Epoch 016/100 | Loss 0.1175 | Win/lose count 10.4/18.5 (-8.1)
Epoch 01

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore' + str(epochs_test) + '.mp4'))

Win/lose count 19.5/3.0. Average score (16.5)
Win/lose count 1.0/0. Average score (8.75)
Win/lose count 12.5/2.0. Average score (9.333333333333334)
Win/lose count 18.0/2.0. Average score (11.0)
Win/lose count 11.0/3.0. Average score (10.4)
Win/lose count 12.5/2.0. Average score (10.416666666666666)
Win/lose count 5.5/0. Average score (9.714285714285714)
Win/lose count 4.0/0. Average score (9.0)
Win/lose count 6.0/0. Average score (8.666666666666666)
Win/lose count 11.5/0. Average score (8.95)
Win/lose count 16.5/3.0. Average score (9.363636363636363)
Win/lose count 14.0/0. Average score (9.75)
Win/lose count 7.0/1.0. Average score (9.461538461538462)
Win/lose count 7.0/0. Average score (9.285714285714286)
Win/lose count 8.5/0. Average score (9.233333333333333)
Win/lose count 5.5/0. Average score (9.0)
Win/lose count 11.5/2.0. Average score (9.029411764705882)
Win/lose count 16.5/3.0. Average score (9.277777777777779)
Win/lose count 17.0/4.0. Average score (9.473684210526315)
Win/lose c

It sounds to work well but sometimes during the test phase the mouse oscillates between 2 postion and I couldn't understand why.

***